# Imports & Extraction depuis l'API



In [105]:
import pandas as pd
import numpy as np
import requests
import io

In [106]:
# URL Parquet de ton dataset
dataset_id = "accidents-corporels-de-la-circulation-millesime"
parquet_url = f"https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/{dataset_id}/exports/parquet"

# Télécharger le Parquet directement
response = requests.get(parquet_url, timeout=60)
if response.status_code != 200:
    raise Exception(f"Erreur téléchargement Parquet: {response.status_code}")


In [107]:
# Lire le contenu Parquet en DataFrame directement depuis le flux mémoire
parquet_data = response.content
accidents_raw = pd.read_parquet(io.BytesIO(parquet_data))

# Construction des Tables :

*   `fait_accidents`
*   `fait_vehicules`
*   `fait_usagers`
*  `dim_geographie`


# `Fait_accidents`



### Pour créer Cette table on part des données brutes, on enrichie plusieurs colonnes et on fait des vérifications de la qualité de données

### Vérification de la donnée brutes

In [108]:
print(f"Valeur Uniques sur la colonne num_acc:{accidents_raw['num_acc'].nunique()}")

Valeur Uniques sur la colonne num_acc:475911


In [109]:
# Checks sur l'année
display(accidents_raw['an'].value_counts().sort_index(),
        accidents_raw['an'].value_counts().sort_index().shape)

,count
an,
2012,62250
2013,58397
2014,59854
2015,58654
2016,59432
2017,60701
2018,57783
2019,58840


(8,)

In [110]:
display(accidents_raw['jour'].value_counts().sort_index(),
        accidents_raw['jour'].value_counts().sort_index().shape)

,count
jour,
01,14603
02,15263
03,15424
04,15866
05,15826
06,16498
07,16244
08,15624
09,16039


(31,)

In [111]:
display(accidents_raw['mois'].value_counts().sort_index(),
        accidents_raw['mois'].value_counts().sort_index().shape)

,count
mois,
01,36639
02,31931
03,37040
04,37912
05,40331
06,44801
07,42463
08,35225
09,43802


(12,)

## Création des colonnes `heure` & `minute` à partir de de la colonne `hrmn`

In [112]:
accidents_raw[['heure', 'minute']] = accidents_raw['hrmn'].str.split(':',
                                                                       expand=True)

In [113]:
display(accidents_raw['heure'].value_counts().sort_index(), accidents_raw['heure'].value_counts().sort_index().shape)

,count
heure,
00,8348
01,6831
02,5803
03,4732
04,4949
05,6898
06,9783
07,20853
08,28507


(24,)

In [114]:
display(accidents_raw['minute'].value_counts().sort_index(), accidents_raw['minute'].value_counts().sort_index().shape)

,count
minute,
00,74802
01,1489
02,875
03,779
04,752
05,16691
06,813
07,908
08,1112


(60,)

## Enrichissement du nom de commune
Dans la donnée brute deux colonnes renvoient la même information `com_name` `nom_com` mais leurs remplissage est différents certains enregistrements sont nuls. Donc on rempli les valeur nulles de l'une par les valeurs remplies de l'autre  

In [115]:
accidents_raw[(~accidents_raw['nom_com'].isna()) & (accidents_raw['com_name'].isna())][['nom_com','com_name']]

,nom_com,com_name
175,Saint-laurent-sur-manoire,None
564,Gieville,None
1764,Saint-gilles-du-mene,None
1806,Male,None
2922,Daumeray,None
...,...,...
474348,Pont-trambouze,None
474535,Martigne-briand,None
474560,Cran-gevrier,None
475369,Tourlaville,None


In [116]:
accidents_raw[(accidents_raw['nom_com'].isna()) & (~accidents_raw['com_name'].isna())][['nom_com','com_name']]

,nom_com,com_name
376,None,Marseille
378,None,Marseille
379,None,Marseille
380,None,Marseille
402,None,Marseille
...,...,...
475473,None,Marseille
475825,None,Marseille
475826,None,Marseille
475860,None,Marseille


In [117]:
accidents_raw.loc[:, 'nom_comune_enr'] = np.where(accidents_raw['nom_com'].isna(),
                                           accidents_raw['com_name'],
                                           np.where(accidents_raw['com_name'].isna(),
                                                    accidents_raw['nom_com'],
                                                    accidents_raw['com_name'])) # Assuming nom_com is preferred when neither is null

# Display the head of the DataFrame with the new column
display(accidents_raw[['nom_com', 'com_name', 'nom_comune_enr']].head())

# Display rows where nom_com was null to verify the new column
display(accidents_raw[accidents_raw['nom_com'].isna()][['nom_com', 'com_name', 'nom_comune_enr']].head())

# Display rows where com_name was null to verify the new column
display(accidents_raw[accidents_raw['com_name'].isna()][['nom_com', 'com_name', 'nom_comune_enr']].head())

,nom_com,com_name,nom_comune_enr
0,Perpignan,Perpignan,Perpignan
1,Saint-denis,Saint-Denis,Saint-Denis
2,Gennevilliers,Gennevilliers,Gennevilliers
3,Fontenay-le-fleury,Fontenay-le-Fleury,Fontenay-le-Fleury
4,Velizy-villacoublay,Vélizy-Villacoublay,Vélizy-Villacoublay


,nom_com,com_name,nom_comune_enr
10,None,None,None
153,None,None,None
329,None,None,None
330,None,None,None
349,None,None,None


,nom_com,com_name,nom_comune_enr
10,None,None,None
153,None,None,None
175,Saint-laurent-sur-manoire,None,Saint-laurent-sur-manoire
329,None,None,None
330,None,None,None


## Enrichissement de la colonne `com` qui correspond au code de la commune.
* La forme du code de la commune est de 5 chiffre normalement, mais sur les données brutes il y'a des valeurs à  3 chiffre qui correspondent au code la commune sans y intégrer le code de département. Pour harmoniser le tout on rajoute le code du département de la colonne `dep`pour avoir un code commune de 5 chiffre partout.   

In [118]:
def enrich_com_code(row):
    if pd.isna(row['com']) or pd.isna(row['dep']):
        return None
    com_str = str(row['com'])
    dep_str = str(row['dep'])

    if len(com_str) < 5:
      if len(com_str) == 2:
        return dep_str + '0' + com_str
      else:
        return dep_str + com_str
    else:
        return com_str

accidents_raw['code_comune_enrichie'] = accidents_raw.apply(enrich_com_code, axis=1)
display(accidents_raw[['com', 'dep', 'code_comune_enrichie']].head())

,com,dep,code_comune_enrichie
0,136,66,66136
1,066,93,93066
2,036,92,92036
3,242,78,78242
4,640,78,78640


In [119]:
accidents_raw[['com','nom_comune_enr','code_comune_enrichie','dep','dep_name']]

,com,nom_comune_enr,code_comune_enrichie,dep,dep_name
0,136,Perpignan,66136,66,Pyrénées-Orientales
1,066,Saint-Denis,93066,93,Seine-Saint-Denis
2,036,Gennevilliers,92036,92,Hauts-de-Seine
3,242,Fontenay-le-Fleury,78242,78,Yvelines
4,640,Vélizy-Villacoublay,78640,78,Yvelines
...,...,...,...,...,...
475906,046,Biziat,01046,01,Ain
475907,314,Pontcharra,38314,38,Isère
475908,003,Acquigny,27003,27,Eure
475909,064,Saint-Cloud,92064,92,Hauts-de-Seine


## Ajout d'une colonne booléenne `en_agglomeration`

In [120]:
accidents_raw['en_agglomeration'] = accidents_raw['agg'].apply(lambda x: True if x == 'En agglomération' else (False if x == 'Hors agglomération' else None))
display(accidents_raw[['agg', 'en_agglomeration']].head())

,agg,en_agglomeration
0,Hors agglomération,False
1,Hors agglomération,False
2,Hors agglomération,False
3,Hors agglomération,False
4,Hors agglomération,False


## Modification des colonne `lan` et `long`

 Pour cette modification, les valeurs brute continnent des virgules et qui sont parsé comme des string, d'autre valeurs n'ont même pas de virgule donc pour harmoniser le tout on prend les 2 premier charactère et on met un point puis on les converti en `float`

In [121]:
def convert_lat_long(coord):
    if pd.isna(coord):
        return None
    coord_str = str(coord)
    if ',' in coord_str:
        return float(coord_str.replace(',', '.'))
    elif len(coord_str) >= 2:
        return float(coord_str[:2] + '.' + coord_str[2:])
    else:
        return None

accidents_raw['lat_float'] = accidents_raw['lat'].apply(convert_lat_long)
accidents_raw['long_float'] = accidents_raw['long'].apply(convert_lat_long)

In [122]:
accidents_raw[(accidents_raw['lat_float']>-90.0) & (accidents_raw['lat_float'] < 90.0)][['num_acc','adr','jour','mois','lat','long','lat_float','long_float']]

,num_acc,adr,jour,mois,lat,long,lat_float,long_float
0,201700033619,COPENHAGUE (ROND POINT D,19,05,4269110,0285200,42.69110,2.85200
3,201700048486,A12 Y,04,05,4880700,0205200,48.80700,2.05200
4,201700048639,N118,11,07,4878430,0222310,48.78430,2.22310
5,201700048644,A86,14,07,4893805,0229921,48.93805,2.29921
7,201700049502,AUTOROUTE A6,04,10,4866250,0236850,48.66250,2.36850
...,...,...,...,...,...,...,...,...
475898,201300016891,"61, SAINTE ANNE (BOULEVA",31,01,4914400,0022119,49.14400,0.22119
475900,201400044296,AV DU MARECHAL FOCH,27,06,4878080,0243754,48.78080,2.43754
475901,201600000965,None,02,09,4789895,-143050,47.89895,-1.43050
475903,201600001483,None,15,11,4934330,0020212,49.34330,0.20212


In [123]:
accidents_raw[(accidents_raw['lat_float']<-90.0) | (accidents_raw['lat_float'] > 90.0)][['num_acc','adr','jour','mois','lat','long','lat_float','long_float']]

,num_acc,adr,jour,mois,lat,long,lat_float,long_float


In [124]:
accidents_raw[(accidents_raw['long_float']>-180.0) & (accidents_raw['long_float'] < 180)][['num_acc',
                                                                                              'adr',
                                                                                              'com',
                                                                                              'nom_comune_enr',
                                                                                              'dep',
                                                                                              'dep_name',
                                                                                              'lat',
                                                                                              'long',
                                                                                              'lat_float',
                                                                                              'long_float']]

,num_acc,adr,com,nom_comune_enr,dep,dep_name,lat,long,lat_float,long_float
0,201700033619,COPENHAGUE (ROND POINT D,136,Perpignan,66,Pyrénées-Orientales,4269110,0285200,42.69110,2.85200
3,201700048486,A12 Y,242,Fontenay-le-Fleury,78,Yvelines,4880700,0205200,48.80700,2.05200
4,201700048639,N118,640,Vélizy-Villacoublay,78,Yvelines,4878430,0222310,48.78430,2.22310
5,201700048644,A86,036,Gennevilliers,92,Hauts-de-Seine,4893805,0229921,48.93805,2.29921
7,201700049502,AUTOROUTE A6,687,Viry-Châtillon,91,Essonne,4866250,0236850,48.66250,2.36850
...,...,...,...,...,...,...,...,...,...,...
475898,201300016891,"61, SAINTE ANNE (BOULEVA",366,Lisieux,14,Calvados,4914400,0022119,49.14400,0.22119
475900,201400044296,AV DU MARECHAL FOCH,028,Créteil,94,Val-de-Marne,4878080,0243754,48.78080,2.43754
475901,201600000965,None,333,Le Theil-de-Bretagne,35,Ille-et-Vilaine,4789895,-143050,47.89895,-1.43050
475903,201600001483,None,578,Saint-Gatien-des-Bois,14,Calvados,4934330,0020212,49.34330,0.20212


In [125]:
accidents_raw[(accidents_raw['long_float']<-180.0) | (accidents_raw['long_float'] > 180)][['num_acc','adr','jour','mois','lat','long','lat_float','long_float']]

,num_acc,adr,jour,mois,lat,long,lat_float,long_float


## Harmonisation de la colonne `lum`

On utilise le mapping donnée par le fichier description des données brutes afin d'harmoniser les données et ne pas avoir des chaîne de caractères

In [126]:
luminosity_mapping = {
    "Plein jour" : 1,
    "Crépuscule ou aube": 2,
    "Nuit sans éclairage public": 3,
    "Nuit avec éclairage public non allumé": 4,
    "Nuit avec éclairage public allumé": 5
}
accidents_raw['luminosite'] = accidents_raw['lum'].map(luminosity_mapping)
display(accidents_raw[['lum', 'luminosite']].value_counts())

,,count
lum,luminosite,
Plein jour,1,323243
Nuit avec éclairage public allumé,5,77740
Nuit sans éclairage public,3,40858
Crépuscule ou aube,2,29770
Nuit avec éclairage public non allumé,4,4300


## Harmonisation de la colonne `atm`

In [127]:
atmospheric_mapping={
    "Normale" : 1,
    "Pluie légère": 2,
    "Pluie forte": 3,
    "Neige - grêle": 4,
    "Brouillard - fumée": 5,
    "Vent fort - tempête": 6,
    "Temps éblouissant": 7,
    "Temps couvert": 8,
    "Autre": 9
}
accidents_raw['conditions_atmospheriques'] = accidents_raw['atm'].map(atmospheric_mapping)
display(accidents_raw[['atm', 'conditions_atmospheriques']].value_counts())

,,count
atm,conditions_atmospheriques,
Normale,1.0,381122
Pluie légère,2.0,52220
Temps couvert,8.0,15449
Pluie forte,3.0,11195
Temps éblouissant,7.0,6239
Brouillard - fumée,5.0,3345
Autre,9.0,2941
Neige - grêle,4.0,2148
Vent fort - tempête,6.0,1201


## Harmonisation de la colonne `int`

In [128]:
def group_intersection_type(intersection_type):
    if int(intersection_type) in [0, 1]:
        return 1
    else:
        return int(intersection_type)

accidents_raw['type_intersection'] = accidents_raw['int'].apply(group_intersection_type)
display(accidents_raw[['int', 'type_intersection']].value_counts().sort_index())

,,count
int,type_intersection,
0,1,90
1,1,328937
2,2,56882
3,3,44672
4,4,8361
5,5,4833
6,6,15386
7,7,6100
8,8,590


## Harmonisation de la colonne `col`

In [129]:
collision_mapping = {
    "Deux véhicules - frontale": 1,
    "Deux véhicules – par l’arrière": 2,
    "Deux véhicules – par le coté": 3,
    "Trois véhicules et plus – en chaîne": 4,
    "Trois véhicules et plus - collisions multiples": 5,
    "Autre collision": 6,
    "-1": 6,
    "Sans collision": 7
}
accidents_raw['type_collision'] = accidents_raw['col'].map(collision_mapping)
display(accidents_raw[['col', 'type_collision']].value_counts().sort_index())

,,count
col,type_collision,
-1,6.0,2
Autre collision,6.0,161298
Deux véhicules - frontale,1.0,45192
Deux véhicules – par le coté,3.0,132605
Deux véhicules – par l’arrière,2.0,58352
Sans collision,7.0,46028
Trois véhicules et plus - collisions multiples,5.0,15508
Trois véhicules et plus – en chaîne,4.0,16916


## Création de la colonne `date`

In [130]:
accidents_raw['date_accident'] = pd.to_datetime(accidents_raw[['an', 'mois', 'jour']].rename(columns={'an': 'year', 'mois': 'month', 'jour': 'day'}))
display(accidents_raw[['an', 'mois', 'jour', 'date_accident']].head())

,an,mois,jour,date_accident
0,2017,05,19,2017-05-19
1,2017,12,25,2017-12-25
2,2017,01,01,2017-01-01
3,2017,05,04,2017-05-04
4,2017,07,11,2017-07-11


In [131]:
print(f"Pourcentation des valeurs nulles : {accidents_raw['date_accident'].isna().sum()/accidents_raw.shape[0]}%" )

Pourcentation des valeurs nulles : 0.0%


## Création de la colonne `jour_semaine`

In [132]:
accidents_raw['jour_semaine'] = accidents_raw['date_accident'].dt.dayofweek + 1
display(accidents_raw[['date_accident', 'jour_semaine']].head())

,date_accident,jour_semaine
0,2017-05-19,5
1,2017-12-25,1
2,2017-01-01,7
3,2017-05-04,4
4,2017-07-11,2


## Selection des colonnes pour la table `fait_accidents`

In [133]:
columns_mapping = {
    'num_acc': 'num_acc',
    'date_accident': 'date_accident',
    'heure' : 'heure',
    'minute' : 'minute',
    'an': 'annee',
    'mois': 'mois',
    'jour': 'jour',
    'jour_semaine': 'jour_semaine',
    'code_comune_enrichie':'com_code',
    'dep': 'departement_code',
    'en_agglomeration': 'en_agglomeration',
    'lat_float': 'latitude',
    'long_float': 'longitude',
    'adr': 'adresse',
    'luminosite': 'luminosite',
    'conditions_atmospheriques': 'conditions_atmospheriques',
    'type_intersection': 'type_intersection',
    'type_collision': 'type_collision'
}
fait_accidents = accidents_raw[list(columns_mapping.keys())].copy().rename(columns=columns_mapping)
display(fait_accidents.head())

,num_acc,date_accident,heure,minute,annee,mois,jour,jour_semaine,com_code,departement_code,en_agglomeration,latitude,longitude,adresse,luminosite,conditions_atmospheriques,type_intersection,type_collision
0,201700033619,2017-05-19,17,40,2017,05,19,5,66136,66,False,42.6911,2.8520,COPENHAGUE (ROND POINT D,1,1.0,6,2.0
1,201700048262,2017-12-25,17,55,2017,12,25,1,93066,93,False,NaN,NaN,AUTOROUTE A1,2,1.0,1,4.0
2,201700048288,2017-01-01,17,25,2017,01,01,7,92036,92,False,NaN,NaN,A86,3,8.0,1,4.0
3,201700048486,2017-05-04,00,04,2017,05,04,4,78242,78,False,48.8070,2.0520,A12 Y,4,9.0,1,6.0
4,201700048639,2017-07-11,08,10,2017,07,11,2,78640,78,False,48.7843,2.2231,N118,1,1.0,1,3.0


# `fait_vehicules`: **Grain -> 1 ligne = 1 véhicule**

Pour cette Table il faut d'abord selectionner les colonnes qui remontent toute les information concernant le(s) véhicule(s) impliqué(s) dans un accident. Ensuite on crée une table du grain indiqué

In [134]:
selected_columns_vehicules = [
    'num_acc',
    'num_veh',
    'senc',
    'catv',
    'obs',
    'obsm',
    'choc',
    'manv',
    'occutc'
    ]
silver_vehicules_stage = accidents_raw[selected_columns_vehicules]

columns_to_split_vehicules = ['num_veh', 'senc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'occutc']
for col in columns_to_split_vehicules:
    silver_vehicules_stage.loc[:, col] = silver_vehicules_stage[col].astype(str).apply(lambda x: x.split(',') if pd.notna(x) else [])

display(silver_vehicules_stage.head())

,num_acc,num_veh,senc,catv,obs,obsm,choc,manv,occutc
0,201700033619,"[B01, A01]",[None],"[VL seul, VL seul]",[None],"[Véhicule, Véhicule]","[Arrière, Avant]",[Sans changement de direction],[None]
1,201700048262,"[A01, E01, C01, B01, D01]",[PK ou PR ou numéro d’adresse postale croissan...,"[VL seul, VL seul, VL seul, VL seul, VL seul]",[None],"[Véhicule, Véhicule, Véhicule, Véhicule, Véhic...","[Avant droit, Arrière, Côté gauche, Arrière dr...","[Manœuvre d’évitement, Arrêté (hors stationnem...",[None]
2,201700048288,"[D01, B01, A01, C01]",[PK ou PR ou numéro d’adresse postale décroiss...,"[VL seul, VL seul, VL seul, VL seul]",[None],"[Véhicule, Véhicule, Véhicule, Véhicule]","[Arrière, Avant, Avant, Arrière]","[Même sens, Même sens, Même sens, Même sens]",[None]
3,201700048486,"[A01, B01]",[PK ou PR ou numéro d’adresse postale croissan...,"[VL seul, VL seul]",[None],"[Véhicule, Véhicule]","[Avant gauche, Côté gauche]","[Changeant de file A gauche, Déporté A gauche]",[None]
4,201700048639,"[Z01, A01]",[PK ou PR ou numéro d’adresse postale décroiss...,"[VL seul, Motocyclette > 125 cm3]",[None],"[Véhicule, Véhicule]",[Avant droit],[Sans changement de direction],[None]


## Explosion de la table selon le nombre de véhicules impacté

In [135]:
silver_vehicules_stage['original_index'] = silver_vehicules_stage.index

exploded_vehicules_dfs = []
columns_to_explode_vehicules = ['num_veh', 'senc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'occutc']

for col in columns_to_explode_vehicules:
    exploded_col_df = silver_vehicules_stage[['original_index', 'num_acc', col]].explode(col)
    exploded_col_df['order'] = exploded_col_df.groupby(['original_index', 'num_acc']).cumcount()
    exploded_col_df = exploded_col_df.rename(columns={col: f'{col}_exploded'})
    exploded_vehicules_dfs.append(exploded_col_df)

silver_vehicules = exploded_vehicules_dfs[0]
for i in range(1, len(exploded_vehicules_dfs)):
    silver_vehicules = pd.merge(silver_vehicules, exploded_vehicules_dfs[i], on=['original_index', 'num_acc', 'order'], how='left')

silver_vehicules = silver_vehicules.drop(columns=['original_index', 'order'])

# Reset the index
silver_vehicules = silver_vehicules.reset_index(drop=True)

display(silver_vehicules.head())

/tmp/ipython-input-66159639.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  silver_vehicules_stage['original_index'] = silver_vehicules_stage.index


,num_acc,num_veh_exploded,senc_exploded,catv_exploded,obs_exploded,obsm_exploded,choc_exploded,manv_exploded,occutc_exploded
0,201700033619,B01,None,VL seul,None,Véhicule,Arrière,Sans changement de direction,None
1,201700033619,A01,NaN,VL seul,NaN,Véhicule,Avant,NaN,NaN
2,201700048262,A01,PK ou PR ou numéro d’adresse postale croissant,VL seul,None,Véhicule,Avant droit,Manœuvre d’évitement,None
3,201700048262,E01,PK ou PR ou numéro d’adresse postale croissant,VL seul,NaN,Véhicule,Arrière,Arrêté (hors stationnement),NaN
4,201700048262,C01,PK ou PR ou numéro d’adresse postale croissant,VL seul,NaN,Véhicule,Côté gauche,Arrêté (hors stationnement),NaN


In [136]:
print(f"Evolution de la dimension de la table véhicules : {silver_vehicules_stage.shape[0]} lignes -> {silver_vehicules.shape[0]} lignes")

Evolution de la dimension de la table véhicules : 475911 lignes -> 811335 lignes


# Tentative de construction de la table `fait_usagers` **Grain -> 1 ligne = 1 usager**

* **Approche**: Pour cette Table il faut selectionner les colonnes qui remontent toute les information concernant le(s) usager(s) impliqué(s) dans un accident. Ensuite on crée une table du grain indiqué

In [137]:
selected_columns_usagers = [
    'num_acc',
    'num_veh',
    'place',
    'catu',
    'grav',
    'sexe',
    'an_nais',
    'trajet',
    'secu',
    'locp',
    'actp',
    'etatp'
    ]
silver_usager_stage = accidents_raw[selected_columns_usagers]

In [138]:
columns_to_split_vehicules = ['num_veh', 'place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu', 'locp', 'actp', 'etatp']
for col in columns_to_split_vehicules:
    silver_usager_stage.loc[:, col] = silver_usager_stage[col].astype(str).apply(lambda x: x.split(',') if pd.notna(x) else [])

display(silver_usager_stage.head())

,num_acc,num_veh,place,catu,grav,sexe,an_nais,trajet,secu,locp,actp,etatp
0,201700033619,"[B01, A01]","[1, 1, 2, 2]","[Conducteur, Conducteur, Passager, Passager]","[Blessé, Blessé, Blessé, Blessé]","[Masculin, Masculin, Masculin, Masculin]","[1996, 1939, 1996, 1931]","[Promenade – loisirs, Promenade – loisirs, Pro...","[Ceinture, Ceinture, Ceinture, Ceinture]",[None],"[Se déplaçant, Se déplaçant, Se déplaçant, Se ...",[None]
1,201700048262,"[A01, E01, C01, B01, D01]","[1, 1, 1, 1, 1]","[Conducteur, Conducteur, Conducteur, Conducteu...","[Indemne, Indemne, Blessé, Indemne, Indemne]","[Masculin, Masculin, Féminin, Féminin, Masculin]","[1952, 1934, 1979, 1986, 1978]",[None],"[Ceinture, Ceinture, Ceinture, Ceinture, Ceint...",[None],"[Se déplaçant, Se déplaçant, Se déplaçant, Se ...",[None]
2,201700048288,"[D01, B01, A01, C01]","[1, 1, 1, 2, 1]","[Conducteur, Conducteur, Conducteur, Passager,...","[Indemne, Indemne, Indemne, Blessé, Indemne]","[Masculin, Masculin, Masculin, Masculin, Mascu...","[1982, 1978, 1990, 2004, 1978]",[None],"[Ceinture, Ceinture, Ceinture, Ceinture, Ceint...",[None],"[Se déplaçant, Se déplaçant, Se déplaçant, Se ...",[None]
3,201700048486,"[A01, B01]","[1, 1, 2]","[Conducteur, Conducteur, Passager]","[Blessé, Indemne, Blessé]","[Masculin, Masculin, Masculin]","[1992, 1990, 1992]",[Autre],"[Ceinture, Ceinture, Ceinture]",[None],"[Se déplaçant, Se déplaçant, Se déplaçant]",[None]
4,201700048639,"[Z01, A01]","[1, 1]","[Conducteur, Conducteur]","[Indemne, Blessé]","[Masculin, Masculin]","[1992, 1960]","[Domicile – travail, Domicile – travail]","[Ceinture, Casque]",[None],"[Se déplaçant, Se déplaçant]",[None]


In [139]:
tentative_usager = pd.merge(
    silver_usager_stage.drop(columns=['num_veh']),
    silver_vehicules[['num_acc','num_veh_exploded']],
    on='num_acc',
    how='left'
)
tentative_usager['original_index'] = tentative_usager.index
tentative_usager = tentative_usager.explode('place')
tentative_usager['order'] = tentative_usager.groupby(['original_index',
                                             'num_veh_exploded',
                                             'place']).cumcount()


display(tentative_usager.head(),tentative_usager.shape)

,num_acc,place,catu,grav,sexe,an_nais,trajet,secu,locp,actp,etatp,num_veh_exploded,original_index,order
0,201700033619,1,"[Conducteur, Conducteur, Passager, Passager]","[Blessé, Blessé, Blessé, Blessé]","[Masculin, Masculin, Masculin, Masculin]","[1996, 1939, 1996, 1931]","[Promenade – loisirs, Promenade – loisirs, Pro...","[Ceinture, Ceinture, Ceinture, Ceinture]",[None],"[Se déplaçant, Se déplaçant, Se déplaçant, Se ...",[None],B01,0,0
0,201700033619,1,"[Conducteur, Conducteur, Passager, Passager]","[Blessé, Blessé, Blessé, Blessé]","[Masculin, Masculin, Masculin, Masculin]","[1996, 1939, 1996, 1931]","[Promenade – loisirs, Promenade – loisirs, Pro...","[Ceinture, Ceinture, Ceinture, Ceinture]",[None],"[Se déplaçant, Se déplaçant, Se déplaçant, Se ...",[None],B01,0,1
0,201700033619,2,"[Conducteur, Conducteur, Passager, Passager]","[Blessé, Blessé, Blessé, Blessé]","[Masculin, Masculin, Masculin, Masculin]","[1996, 1939, 1996, 1931]","[Promenade – loisirs, Promenade – loisirs, Pro...","[Ceinture, Ceinture, Ceinture, Ceinture]",[None],"[Se déplaçant, Se déplaçant, Se déplaçant, Se ...",[None],B01,0,0
0,201700033619,2,"[Conducteur, Conducteur, Passager, Passager]","[Blessé, Blessé, Blessé, Blessé]","[Masculin, Masculin, Masculin, Masculin]","[1996, 1939, 1996, 1931]","[Promenade – loisirs, Promenade – loisirs, Pro...","[Ceinture, Ceinture, Ceinture, Ceinture]",[None],"[Se déplaçant, Se déplaçant, Se déplaçant, Se ...",[None],B01,0,1
1,201700033619,1,"[Conducteur, Conducteur, Passager, Passager]","[Blessé, Blessé, Blessé, Blessé]","[Masculin, Masculin, Masculin, Masculin]","[1996, 1939, 1996, 1931]","[Promenade – loisirs, Promenade – loisirs, Pro...","[Ceinture, Ceinture, Ceinture, Ceinture]",[None],"[Se déplaçant, Se déplaçant, Se déplaçant, Se ...",[None],A01,1,0


(1937873, 14)